In [8]:
import numpy as np
from netCDF4 import Dataset
from datetime import datetime

class gsidiag:
    
    def __init__(self, path):
        """
        Initialize a GSI diagnostic object
        INPUT:
            path : path to GSI diagnostic object
        """

        self.path = path
        
    def __len__(self):
        return len(self.lats)        
        
    def get_metadata(self):
        """
        Grabs metadata from the diagnostic filename **NEED TO ADDRESS STILL**
        """
        
        dtype = self.path.split('/')[-1].split('.')[0].split('_')[1]
        str_date = self.path.split('/')[-1].split('.')[1]
        date = datetime.strptime(str_date, '%Y%m%d%H')
        ftype = self.path.split('/')[-1].split('.')[0].split('_')[-1]
        
        self.dtype = dtype
        self.date = date
        self.ftype = ftype
        
    def query_dataType(self, dtype, idx):
        """
        Query the data type being requested and returns
        the appropriate indexed data
        """
        if dtype == 'O-F':
            data = self.omf[idx]
            return data
        
        elif dtype == 'observation':
            data = self.observation[idx]
            return data
        
        elif dtype == 'O-A':
            # Not sure if I need to do this because 'ges' and 'anl' files are both f.variable['Obs_Minus_Forecast_adjusted'][:]
            # but I will leave it for now
            check = check_o_minus_a()
            if check:
                data = self.omf[idx]
                return data
            else:
                print('File type does not support O-A')
        
        elif dtype == 'H(x)':
            Hx = get_H_x()
            data = Hx[idx]
            return data
        
        else:
            raise Exception(f'Unrecognizable data type: {dtype}')
            return None
            
        
    def check_o_minus_a(self):
        """
        Checks if the diasnostic file is an analyses file.
        """
        
        ftype = self.path.split('/')[-1].split('.')[0].split('_')[-1]
        
        if ftype == 'anl':
            return True
        else:
            return False
        
    def get_H_x(self):
        """
        Calculates H(x): O - (O-F) = F = H(x)
        """
        
        Hx = self.o - self.omf
        return Hx        
        
    
class conventional(gsidiag):
    
    def __init__(self, path):
        """
        Initialize a conventional GSI diagnostic object
        INPUT:
            path   : path to conventional GSI diagnostic object
        RESULT:
            self   : GSIdiag object containing the path to extract data
        """
        super().__init__(path)
        
        self.read_conv_obs()
        
    def __str__(self):
        return "Conventional object"
        
    def read_conv_obs(self):
        """
        Reads the data from the conventional diagnostic file during initialization.
        """
        
        f = Dataset(self.path, mode='r')
        
        self.o_type = f.variables['Observation_Type'][:]
        self.lons = f.variables['Longitude'][:]
        self.lats = f.variables['Latitude'][:]
        self.press = f.variables['Pressure'][:]
        self.time = f.variables['Time'][:]
        self.stnelev = f.variables['Station_Elevation'][:]
        self.o = f.variables['Observation'][:]
        
        if self.path.split('/')[-1].split('.')[0].split('_')[2] == 'uv':
            self.u_o = f.variables['u_Observation'][:]
            self.v_o = f.variables['v_Observation'][:]
            
            self.u_omf = f.variables['u_Obs_Minus_Forecast_adjusted'][:]
            self.v_omf = f.variables['v_Obs_Minus_Forecast_adjusted'][:]
            
        else:
            self.o = f.variables['Observation'][:]
            self.omf = f.variables['Obs_Minus_Forecast_adjusted'][:]
        
        f.close()

            
    def getData(self, dtype, obsid=None, qcflag=None):
        """
        Given parameters, get the data from a conventional diagnostic file
        INPUT:
            dtype  : type of data to extract i.e. observation, O-F, O-A, H(x)
            obsid  : observation measurement ID number
            qcflag : qc flag (default: None) i.e. 0, 1
        OUTPUT:
            data   : requested data
        """
        
        idx = self.get_idx_conv(obsid, qcflag=None)
        
        data = self.query_dataType(dtype, idx)
        
        return data
    
    
    def get_idx_conv(self, obsid=None, qcflag=None):
        """
        Given parameters, get the indices of the observation
        locations from a conventional diagnostic file
        INPUT:
            obsid  : observation measurement ID number
            qcflag : qc flag (default: None) i.e. 0, 1
        OUTPUT:
            idx    : indices of the requested data in the file
        """
        
        if not obsid:
            obsid=None
        if not qcflag:
            qcflag=None
        
        idx = self.o_type
        
        if obsid != None:
            valid_idxs = np.isin(idx, obs_id)
            idx = np.where(valid_idxs)
        if qcflag != None:
            valid_idxs = np.isin(idx, qcflag)
            idx = np.where(valid_idxs)
        
        return idx
    
    def get_lat_lon(self, obsid=None, qcflag=None):
        """
        Gets lats and lons with desired indices
        """
        idx = get_idx_conv(obsid, qcflag)
        return self.lats[idx], self.lons[idx]
    
    
class satellite(gsidiag):
    
    def __init__(self, path):
        """
        Initialize a satellite GSI diagnostic object
        INPUT:
            path   : path to conventional GSI diagnostic object
        RESULT:
            self   : GSIdiag object containing the path to extract data
        """
        super().__init__(path)

        self.read_satellite_obs()
        
    def __str__(self):
        return "Satellite object"
        
    def read_satellite_obs(self):
        """
        Reads the data from the satellite diagnostic file during initialization.
        """
        
        f = Dataset(self.path, mode='r')
        
        self.channel_idx = f.variables['Channel_Index'][:]
        self.lons = f.variables['Longitude'][:]
        self.lats = f.variables['Latitude'][:]
        self.time = f.variables['Obs_Time'][:]
        self.o = f.variables['Observation'][:]
        self.omf = f.variables['Obs_Minus_Forecast_adjusted'][:]
        self.qc_flag = f.variables['QC_Flag'][:]
        self.water_frac = f.variables['Water_Fraction'][:]
        self.land_frac = f.variables['Land_Fraction'][:]
        self.ice_frac = f.variables['Ice_Fraction'][:]
        self.snow_frac = f.variables['Snow_Fraction'][:]
        
        f.close()
        
        
    def getData(self, dtype, channel=None, qcflag=None):
        """
        Given parameters, get the data from a satellite 
        diagnostic file.
        INPUT:
            dtype   : type of data to extract i.e. observation, O-F, O-A, H(x)
            channel : observation channel number
            qcflag  : qc flag (default: None) i.e. 0, 1
        OUTPUT:
            data    : requested data
            
        """
        
        idx = self.get_idx_sat(channel, qc_flag)
        
        data = self.query_dataType(dtype, idx)
        
        return data
    
    def get_idx_sat(self, channel=None, qcflag=None):
        """
        Given parameters, get the indices of the observation
        locations from a satellite diagnostic file.
        """
        
        if not channel:
            channel=None
        if not qcflag:
            qcflag=None
            
        idx = self.channel_idx
        
        if channel != None:
            valid_idxs = np.isin(idx, channel)
            idx = np.where(valid_idxs)
        if qcflag != None:
            valid_idxs = np.isin(self.qc_flag[idx], qcflag)
            idx = np.where(valid_idxs)
            
        return idx
    
    # How can we get this method to be used for both conv
    # and satellite
    def get_lat_lon(self, channel=None, qcflag=None):
        """
        Gets lats and lons with desired indices
        """
        idx = self.get_idx_sat(channel, qcflag)
        return self.lats[idx], self.lons[idx]
        
########################################################    

path = '/scratch1/NCEPDEV/stmp2/Cory.R.Martin/ICs_tmp/v16para/gdas.20200531/00/'
# nc_file = path+'diag_conv_t_ges.2020053100.nc4'
nc_file = path+'diag_amsua_metop-a_ges.2020053100.nc4'

obs_id  = []

channel = [1]
qc_flag = [0]

# DATA_TYPE can be: 'O-F', 'O-A', 'observation', 'H(x)'  (so far)
DATA_TYPE = 'O-F'

if nc_file.split('/')[-1].split('.')[0].split('_')[1] == 'conv':
    # Create Conventional Diagnostic data object
    diag = conventional(nc_file)
    
    print(len(diag))

    # Get data from specified DATA_TYPE, include specified 
    # observation ids and qc flags (Don't know if there are any for conventional data)
    data = diag.getData(DATA_TYPE, obs_id, qc_flag)
    
else:
    # Create Satellite Diagnostic data object
    diag = satellite(nc_file)
    
    print(diag)
    print(len(diag))
    
    data = diag.getData(DATA_TYPE, channel, qc_flag)
    
    lat, lon = diag.get_lat_lon()
    


# data
# diag.__dict__


Satellite object
134325


In [13]:
import numpy as np

y ='Testing...'
x = np.arange(0,10)

print(f"I am {y}" + ', '.join(f"{i}" for i in x))

I am Testing...0, 1, 2, 3, 4, 5, 6, 7, 8, 9


In [3]:
from netCDF4 import Dataset

path = '/scratch1/NCEPDEV/stmp2/Cory.R.Martin/ICs_tmp/v16para/gdas.20200531/00/'
nc_file = path+'diag_amsua_metop-a_ges.2020053100.nc4'

f = Dataset(nc_file, mode='r')
# print(f)

chan = f.variables['Channel_Index'][:]
f.close()

print(chan)


[ 1  2  3 ... 13 14 15]


In [ ]:
def read_satellite_obs(self):
        """
        Reads the data from the conventional diagnostic file during initialization.
        """
        
        f = Dataset(self.path, mode='r')
        
        self.channel_idx = f.variables['Channel_Index'][:]
        self.lons = f.variables['Longitude'][:]
        self.lats = f.variables['Latitude'][:]
        self.time = f.variables['Obs_Time'][:]
        self.o = f.variables['Observation'][:]
        self.omf = f.variables['Obs_Minus_Forecast_adjusted'][:]
        self.qc_flag = f.variables['QC_Flag'][:]
        self.water_frac = f.variables['Water_Fraction'][:]
        self.land_frac = f.variables['Land_Fraction'][:]
        self.ice_frac = f.variables['Ice_Fraction'][:]
        self.snow_frac = f.variables['Snow_Fraction'][:]
        
        f.close()

In [5]:
path = '/scratch1/NCEPDEV/stmp2/Cory.R.Martin/ICs_tmp/v16para/gdas.20200531/00/'
nc_file = path + 'diag_conv_uv_ges.2020053100.nc4'

f = Dataset(nc_file, mode='r')
print(f)

f.close()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    date_time: 2020053100
    dimensions(sizes): nobs(1725950), Station_ID_maxstrlen(8), Observation_Class_maxstrlen(7)
    variables(dimensions): |S1 Station_ID(nobs,Station_ID_maxstrlen), |S1 Observation_Class(nobs,Observation_Class_maxstrlen), int32 Observation_Type(nobs), int32 Observation_Subtype(nobs), float32 Latitude(nobs), float32 Longitude(nobs), float32 Station_Elevation(nobs), float32 Pressure(nobs), float32 Height(nobs), float32 Time(nobs), float32 Prep_QC_Mark(nobs), float32 Setup_QC_Mark(nobs), float32 Prep_Use_Flag(nobs), float32 Analysis_Use_Flag(nobs), float32 Nonlinear_QC_Rel_Wgt(nobs), float32 Errinv_Input(nobs), float32 Errinv_Adjust(nobs), float32 Errinv_Final(nobs), float32 Wind_Reduction_Factor_at_10m(nobs), float32 u_Observation(nobs), float32 u_Obs_Minus_Forecast_adjusted(nobs), float32 u_Obs_Minus_Forecast_unadjusted(nobs), float32 v_Observation(nobs), float32 v_Obs_Minus_Fo

In [62]:
print(valid_idxs)

[False False False ... False False False]


In [67]:
x = False
if x:
    print('yes')
else: print('no')

no
